In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.utils.vis_utils import plot_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import ExtraTreesClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [75]:
train = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv",index_col="row_id")
test = pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv",index_col="row_id")

# Exploring the Data

The data comes from DNA readings using a spectroscopy technique. A DNA sequence is a finite sequence of letters "A","T","G" and "C" (a bacteria genome has length around 5 million letters). They take a sequence and sample it in subsequences of length 10 (10-mers) and count how many of each letter appears in that subsequence. 

For example: $ATGCAATGCG \to A_3 T_2 G_3 C_2$

This means instead of storing the whole sequence they just store how many of each possible subsequence of length 10.

A few other things also happen in the data:

1.  Random noise is added to these counts which is meant to correspond to mutations, the possibility the same subseuquence is not read completely/read multiple times, etc... Some 10-mers are changed to random values with some error rate.
2. They subtract a bias term so that the entries in the data correspond to the deviation from a random 10-mer

### First look at the data
We need to get an idea of what the data looks like, especially any differences in the training and test sets.

In [76]:
train.head()

In [77]:
print("Training data: ")
train.describe().transpose()[["mean","std","min","max"]]

In [78]:
print("Testing data: ")
train.describe().transpose()[["mean","std","min","max"]]

The values don't look continuous, despite the noise/simulated errors. 

In [79]:
print("The number of unique values in each column is \n")
print(str(train.nunique()))

In [80]:
print("The number of duplicated rows in the training data is", train.duplicated().sum())
print("The total number of rows is", train.shape[0])

It isn't really clear why there are duplicates: is it the sampling method (and so useful data) or is it caused by how they simulate error (so shouldn't really be taken into account if the model is to perform well on real data). Removing duplicates could speed up training significantly, but we don't want to lose any info. It's hard to say what the best move here is given we don't know how the distributions of the training data and testing data differ. 

In [81]:
print("The number of duplicated rows in the testing data is", test.duplicated().sum())
print("The total number of rows is", test.shape[0])

### Bacteria in the training set
Let's look at what bacteria we have in the data set and the proportion of each one.

In [82]:
bacteria = sorted(train["target"].unique())
print("The bacteria in the data set are: ",bacteria)

In [83]:
bacteria_counts = train.groupby("target")["target"].count()
print("Percentage of training set for each bacteria: ")
print(100*bacteria_counts/bacteria_counts.sum())

### Displaying some spectra
It could be fun to plot some of these spectra, so lets pick some random ones. I wouldn't say this is essential, but its a bit of practice.

In [84]:
# Plot some histograms representing some data points
(n_x,n_y) = (3,3)

number_to_plot = n_x*n_y
samples = train.sample(number_to_plot)

fig, axs = plt.subplots(n_x,n_y,sharey=True)
x = 1 + np.arange(286)

for i in range(n_x):
    for j in range(n_y):
        axs[i,j].bar(x,samples.values[i*j][0:286],width=1.5)
        axs[i,j].title.set_text("bacteria: " + samples.values[i*j][286])
fig.set_size_inches(3*n_x,3*n_y)
fig.tight_layout()

It might actually be useful to look at visual features of different rows labelled as the same bacteria.

In [85]:
# Now plot some histograms for some points which are in the same class
plot_bacteria = "Campylobacter_jejuni"
(n_x,n_y) = (3,3)

number_to_plot = n_x*n_y
bacteria_samples = train[train["target"] == plot_bacteria].sample(number_to_plot)
fig, axs = plt.subplots(n_x,n_y,sharey=True)
x = 1 + np.arange(286)

for i in range(n_x):
    for j in range(n_y):
        axs[i,j].bar(x,bacteria_samples.values[i*j][0:286],width=1.5)
        axs[i,j].title.set_text("bacteria: " + bacteria_samples.values[i*j][286])
fig.set_size_inches(3*n_x,3*n_y)
fig.tight_layout()

It looks like they have the same shape but some are scaled by some factor. This is probably to do with the way the error is added, but I haven't read that part of the paper in too much detail.

# Preprocessing
Our preproccessing has a number of steps:
1. (Optional) Remove duplicate columns 
2. Encode each bacteria as a number 0-9
3. Split data into training/validation data
4. Scale the training data so that is has mean 0 and std 1
5. Scale the validation and testing data by the mean/std computed above

Note on removing duplicates: This may change the distribution of bacteria in the training set (remember they were 10% each). Probably unwise to do this unless an extra weight column is added representing the number of duplicates.

In [86]:
def process_data(train,test,drop_duplicates=False):
    X_train = train.copy()
    
    # Drop duplicate columns
    if drop_duplicates == True:
        X_train = X_train.drop_duplicates()
    
    y_train = X_train.pop("target")
    
    # Encode target column with labels 0-9
    labeler = LabelEncoder()
    labeler.fit(y_train)
    y_train_encoded = labeler.transform(y_train)
    
    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train,y_train_encoded,test_size = 0.2, train_size = 0.8)
    
    # Scale data
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    
    # Apply appropriate transformations to testing data so that predictions can be made
    X_test = scaler.transform(test)
    
    return X_train,X_val,X_test,y_train,y_val,labeler
    

In [87]:
X_train,X_val,X_test,y_train,y_val,labeler = process_data(train,test,drop_duplicates=False)

In [88]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

# Creating a model
### Neural Network
One of the models they use in the paper is a Neural Network (among others). This probably won't out perform tree based methods for this type of data, but just for fun let's see what kind of performance we get. 

They say they use "default parameters" and a 100 layer (!) network. This is definitely a typo, they use the SKLearn defaults which are 2 layers of 100 units each. 



In [89]:
def createModelNN(num_layers=2,num_units=256,batch_norm=True,dropout=0.3):
    model = tf.keras.Sequential()
    model.add( tf.keras.layers.Input(X_train.shape[1]) )
    
    for i in range(num_layers):
        model.add( tf.keras.layers.Dense(units=num_units) )
        if batch_norm == True:
            model.add( tf.keras.layers.BatchNormalization() )
        if dropout > 0:
            model.add( tf.keras.layers.Dropout(dropout) )
        model.add( tf.keras.layers.Activation("relu") )
        
    model.add( tf.keras.layers.Dense(units=10,activation="softmax") )
    
    return model

We can then build our model so that it will be ready for training. We also need to define the optimizer and loss function for training the model.

We use the following:

* Number of layers: 2
* Number of units: 256
* Dropout on with a dropping probability of 0.3
* No batch normalization
* Optimizer: ADAM 
* Loss: "sparse_categorical_crossentropy" is the option in Keras which deals with multiclass clasification with our label type

In [90]:
model_NN = createModelNN(num_layers=2,num_units=256,batch_norm=False,dropout=0.3)
model_NN.compile(optimizer="adam", loss = "sparse_categorical_crossentropy",metrics=["accuracy"])
model_NN.build()
print(model_NN.summary())
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

We will train the model with early stopping:

In [91]:
cb = tf.keras.callbacks.EarlyStopping(monitor="val_loss",min_delta=0.01,patience=10)

history = model_NN.fit(X_train,y_train,epochs=100,batch_size=64,validation_data=(X_val,y_val),callbacks=[cb])

To get an idea of how the training went let's plot its progress through iterations of the training process: 

In [92]:
history_df = pd.DataFrame(history.history)
history_df[["accuracy","val_accuracy"]].plot().set(xlabel="Epoch",ylabel="Accuracy")
history_df[["loss","val_loss"]].plot().set(xlabel="Epoch",ylabel="Loss")

### Extra Trees Classifier
The best performing decision tree algorithm they use in their paper is an Extra Trees Classifier. Let's see what performance we can get from this model. We use 500 estimators in the model and default parameters otherwise. 

In [93]:
model_ET = ExtraTreesClassifier(n_estimators=500,verbose=1)
model_ET.fit(X_train,y_train)

# Model Analysis
To analyze each model we will look at the distribution of bacteria predicted as well as which bacteria are mislabelled in the validation set. 
### Neural Network
Let's first visualize which bacteria are mislabelled by the model in the training and validation sets.

In [94]:
val_predictions_NN = model_NN.predict(X_val)
val_predictions_NN = tf.argmax(val_predictions_NN,axis=1)
train_predictions_NN = model_NN.predict(X_train)
train_predictions_NN = tf.argmax(train_predictions_NN,axis=1)

In [95]:
# Get an array of the wrong predictions
wrong_val_NN = np.where(y_val != val_predictions_NN, 1,0)
wrong_train_NN = np.where(y_train != train_predictions_NN,1,0)
print("Number of incorrect predictions in the validation set is ", wrong_val_NN.sum(), " Fraction incorrect is ", wrong_val_NN.sum()/len(wrong_val_NN))
print("Number of incorrect predictions in the training set is ", wrong_train_NN.sum(), " Fraction incorrect is ", wrong_train_NN.sum()/len(wrong_train_NN))

In [96]:
ConfusionMatrixDisplay(confusion_matrix(y_val,val_predictions_NN)).plot()

In [97]:
ConfusionMatrixDisplay(confusion_matrix(y_train,train_predictions_NN)).plot()

Next we can use the model to predict the labels for the test set.

In [98]:
soft_max_predictions_NN = model_NN.predict(X_test)
predictions_numerical_NN = tf.argmax(input=soft_max_predictions_NN, axis=1)
predictions_NN = labeler.inverse_transform(predictions_numerical_NN)

We don't know the distribution of bacteria in the testing set, but perhaps they are equally distributed as in the training data. We can check what the distribution of predicted labels is at least:

In [99]:
test_counts_NN = pd.DataFrame(predictions_NN,columns=["target"]).groupby("target")["target"].count()
print("Percentage of predictions in the test set for each bacteria: ")
print(100*test_counts_NN/test_counts_NN.sum())

Predictions don't look too far off, although we might be under/overpredicting some classes. 

### Extra Trees

We'll repeat the same proccess here to get an idea of the differences of each model. First we look at the predictions on the validation/training set:

In [100]:
val_predictions_ET = model_ET.predict(X_val)
train_predictions_ET = model_ET.predict(X_train)
val_accuracy_ET = model_ET.score(X_val,y_val)
train_accuracy_ET = model_ET.score(X_train,y_train)

In [101]:
wrong_val = np.where(y_val != val_predictions_ET, 1,0)
print("Number of incorrect predictions in the validation set is ", wrong_val.sum(), " Accuracy is ", val_accuracy_ET)
wrong_train = np.where(y_train != train_predictions_ET,1,0)
print("Number of incorrect predictions in the training set is ", wrong_train.sum(), " Accuracy is ", train_accuracy_ET)

In [102]:
ConfusionMatrixDisplay(confusion_matrix(y_val,val_predictions_ET)).plot()

In [103]:
ConfusionMatrixDisplay(confusion_matrix(y_train,train_predictions_ET)).plot()

Now we look at the distribution of predictions of the testing data:

In [104]:
predictions_numerical_ET = model_ET.predict(X_test)
predictions_ET = labeler.inverse_transform(predictions_numerical_ET)

In [105]:
test_counts_ET = pd.DataFrame(predictions_ET,columns=["target"]).groupby("target")["target"].count()
print("Percentage of predictions in the test set for each bacteria: ")
print(100*test_counts_ET/test_counts_ET.sum())

Assuming the test data are distributed equally (which seems to be the case) we are probably under predicting E. coli.

# Submission
To finish off we will export our predictions so that they can be submitted to the competition for ranking. 

In [106]:
submission_NN = pd.DataFrame(predictions_NN, columns=["target"],index=test.index)
submission_NN.to_csv("submissionNN.csv",index=True)

In [107]:
submission_ET = pd.DataFrame(predictions_ET, columns=["target"],index=test.index)
submission_ET.to_csv("submissionET.csv",index=True)